In [1]:
import os
import csv
import cv2
from skimage.feature import hog
from sklearn.externals import joblib

In [2]:
# ucitavanje skupova

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
    return images 

def load_sets():
    sa_knjigom = load_images_from_folder("skup_podataka/saKnjigom")
    bez_knjige = load_images_from_folder("skup_podataka/bezKnjiga")
    test_skup = load_images_from_folder("skup_podataka/test")
    return sa_knjigom, bez_knjige, test_skup

In [3]:
# odredjivanje da li naslici ima knjiga ili nema

def hog_processing(sa_knjigama, bez_knjiga):
    lista_svih = []
    labele = []
    for slika in sa_knjigama:
        features = hog(slika, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2")
        lista_svih.append(features)
        labele.append(1)

    for slika in bez_knjiga:
        features = hog(slika, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2")
        lista_svih.append(features)
        labele.append(0)

    clf = joblib.load("pedestrian.pkl")
    clf.fit(lista_svih, labele)
    return clf

def read_resault(ime_fajla):
    resenje = []
    with open(ime_fajla, encoding="utf8", newline='\n') as f:
        lines = csv.reader(f, delimiter=',')
        for row in lines:
            if (row[0] == "br_knjiga"):
                continue
            niz = []
            if len(row) == 2:
                niz = row[1]
                niz = niz[1:-1]
                niz = niz.split(';')
            resenje.append([int(row[0]), niz])
    return resenje

def predict_books(clf, test_skup, informacije):
    resenje = [i[0] != 0 for i in informacije]

    dobijeni_rezultat = []
    ind = 0
    br_tacnih = 0
    for slika in test_skup:
        features = hog(slika, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2")
        listica = [features, features]
        rez = clf.predict(listica)
        dobijeni_rezultat.append(rez[0])
        if (dobijeni_rezultat[ind] == resenje[ind]):
            br_tacnih += 1
        ind += 1
    tacnost = (br_tacnih / len(resenje)) * 100
    tacnost = round(tacnost, 2)
    return tacnost, dobijeni_rezultat

In [ ]:
# GLAVNA FUNKCIJA

with_books, without_books, test_set = load_sets()
print("Slike sa knjigama:", len(with_books))
print("Slike bez knjiga:", len(without_books))
print("Slike za test:", len(test_set))

print("\nDetekcija da li na slikama ima knjiga ili ne je u toku...")
print("Detekcija moze potrajati oko 10 minuta")
linear_svc = hog_processing(with_books, without_books)
file_info = read_resault('skup_podataka/test_podaci.csv')
accuracy, resault = predict_books(linear_svc, test_set, file_info)
print("\nDetekcija je zavrsena")
print("Tacnost resenja: " + str(accuracy) +"%")

Slike sa knjigama: 210
Slike bez knjiga: 210
Slike za test: 150

Detekcija da li na slikama ima knjiga ili ne je u toku...
Detekcija moze potrajati oko 10 minuta


D:\soft_vezbe\sc-2021-siit\env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[LibLinear]

In [ ]:
import numpy as np
# detekcija slova
def resize_region(region):
    return cv2.resize(region, (28, 28), interpolation=cv2.INTER_NEAREST)

def select_roi(image_orig, image_bin):                                 # roi - region od interesa
    contours, hierarchy = cv2.findContours(image_bin.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    sorted_regions = []                                                # lista sortiranih regiona po X osi
    regions_array = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)                         # koordinate i velicina granicnog pravougaonika
        area = cv2.contourArea(contour)
        if area > 100 and h < 100 and h > 40 and w > 5:
            # kopirati [y:y+h+1, x:x+w+1] sa binarne slike i smestiti u novu sliku
            # oznaciti region pravougaonikom na originalnoj slici sa rectangle funkcijom
            region = image_bin[y:y+h+1, x:x+w+1]
            regions_array.append([resize_region(region), (x, y, w, h)])
            cv2.rectangle(image_orig, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    regions_array = sorted(regions_array, key=lambda x: x[1][0])
    sorted_regions = [region[0] for region in regions_array]
    return image_orig, sorted_regions

def scale_to_range(image):
    return image/255

def matrix_to_vector(image):
    return image.flatten()

def prepare_for_ann(regions):
    ready_for_ann = []
    for region in regions:
        scale = scale_to_range(region)
        ready_for_ann.append(matrix_to_vector(scale))
    return ready_for_ann

def convert_output(alphabet):
    nn_outputs = []
    for index in range(len(alphabet)):
        output = np.zeros(len(alphabet))
        output[index] = 1
        nn_outputs.append(output)
    return np.array(nn_outputs)

def create_ann(output_size):
    ann = Sequential()
    ann.add(Dense(128, input_dim=784, activation='sigmoid'))
    ann.add(Dense(output_size, activation='sigmoid'))
    return ann

def train_ann(ann, X_train, y_train, epochs):
    X_train = np.array(X_train, np.float32) # dati ulaz
    y_train = np.array(y_train, np.float32) # zeljeni izlazi na date ulaze
    
    print("\nTraining started...")
    sgd = SGD(learning_rate=0.01, momentum=0.9)
    ann.compile(loss='mean_squared_error', optimizer=sgd)
    ann.fit(X_train, y_train, epochs=epochs, batch_size=1, verbose=0, shuffle=False)
    print("\nTraining completed...")
    return ann

def winner(output):
    return max(enumerate(output), key=lambda x: x[1])[0]

def display_result(outputs, alphabet):
    result = []
    for output in outputs:
        result.append(alphabet[winner(output)])
    return result

In [ ]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation

from tensorflow.keras.optimizers import SGD

def binarizuj_sliku(siva_slika):
    visina, sirina = siva_slika.shape[0:2]
    bin_slika = np.ndarray((visina, sirina), dtype=np.uint8)                           # mozda visak
    ret, binarna_slika = cv2.threshold(siva_slika, 127, 255, cv2.THRESH_BINARY)
    return binarna_slika
def prikaz_slike(slika, color=False):
    if color:
        plt.imshow(slika)
    else:
        plt.imshow(slika, 'gray')

alfabet_slika = cv2.cvtColor(cv2.imread('bezKvacica.png'), cv2.COLOR_BGR2RGB)
slika = binarizuj_sliku(cv2.cvtColor(alfabet_slika, cv2.COLOR_RGB2GRAY))
selektovani_regioni, simboli = select_roi(alfabet_slika.copy(), slika)
print("Broj prepoznatih regiona: ", len(simboli))
plt.rcParams['figure.figsize'] = [100, 50]
prikaz_slike(selektovani_regioni)
#prikaz_slike(selektovani_regioni)

alfabet = ['A','B','V','G','D','E','Z','I','J','K','L','M','N','O','P','R','S','T','U','F','H','C']
print(len(alfabet))
ulazi = prepare_for_ann(simboli)
izlazi = convert_output(alfabet)
nm = create_ann(22)                          # br slova
nm = train_ann(nm, ulazi, izlazi, epochs=2000)


In [ ]:
# slika_u_boji = ucitaj_sliku('images/slozen_izraz.png')
slika = binarizuj_sliku(test_set[113])
selektovani_regioni, simboli = select_roi(test_set[113].copy(), slika)
print("Broj prepoznatih regiona: ", len(simboli))
prikaz_slike(selektovani_regioni)

ulazi = prepare_for_ann(simboli)
rezultati = nm.predict(np.array(ulazi, np.float32))
izraz = ""
for i in range(len(simboli)):
    maxi = max(rezultati[i])
    for ind in range(len(rezultati[i])):
        if rezultati[i][ind] == maxi:
            izraz += alfabet[ind] + " "
            break
print("Izraz je izracunat: " + izraz)

In [ ]:
"""
    file_info                                                -> [[0, []], [2, ['rec', 'rec', 'rec']], ...]
    
    knjiga_info_7 = file_info[6]                             -> [2, ['rec', 'rec', 'rec']]                -> ima knjiga
    sve_reci_za_ocr = file_info[6][1] = knjiga_info_7[1]     -> ['rec', 'rec', 'rec']
    knjiga_info_5 = file_info[4]                             -> [0, []]                                   -> nema knjiga
    
    --------------------------------------------------------------------------------------------------------------------
    result                                                   -> [0, 1, 0, 1, 1, ...]
    --------------------------------------------------------------------------------------------------------------------
    uslov za ocr:
        => result[ind] == 1 and file_info[ind][0] != 0
    
"""